In [1]:
%load_ext pydough.jupyter_extensions

In [2]:
import pydough
import datetime

import pandas as pd
from pandas.testing import assert_frame_equal, assert_series_equal
import re
import dfcompare

import collections
import numpy as np
import sqlite3 as sql
import os

# Setup demo metadata
pydough.active_session.load_metadata_graph("../metadata/tpch_demo_graph.json", "TPCH");
pydough.active_session.connect_database("sqlite", database="../../tpch.db");
pydough.active_session.metadata

db_path = "../../tpch.db" # variable to pass the path in python function

connection = sql.connect("../../tpch.db")

# Avoid scientific notation
pd.options.display.float_format = '{:.6f}'.format

# TPCH Testing New Queries

The idea of this notebook is to create new ideas in the TPCH business context. Then, we proceed to create their respective SQL query, to try to perform it in PyDough. 

## Bussines Context 1

This consultation aims to identify which three regions generate the highest total sales and who are the main suppliers contributing to those sales. The objective is to assess sales performance by geographic region and to understand which suppliers dominate each region.

Total sales is defined as the extended price times one less discount.

SQL:

In [11]:
query = '''
SELECT
    R_NAME AS region_name,
    S_NAME AS supplier_name,
    SUM(L_EXTENDEDPRICE * (1 - L_DISCOUNT)) AS total_sales
FROM
    REGION
JOIN
    NATION ON R_REGIONKEY = N_REGIONKEY
JOIN
    SUPPLIER ON N_NATIONKEY = S_NATIONKEY
JOIN
    PARTSUPP ON S_SUPPKEY = PS_SUPPKEY
JOIN
    LINEITEM ON PS_PARTKEY = L_PARTKEY AND PS_SUPPKEY = L_SUPPKEY
GROUP BY
    R_NAME, S_NAME
ORDER BY
    total_sales DESC
LIMIT 3;
'''

sql_output = pd.read_sql_query(query, connection)
sql_output

,region_name,supplier_name,total_sales
0,AFRICA,Supplier#000005994,28698575.631300
1,EUROPE,Supplier#000006490,28579287.000400
2,MIDDLE EAST,Supplier#000005991,28542342.998400


PyDough solution:

In [ ]:
%%pydough

supplier = suppliers(region_name=nation.region.name, supplier_name=name)

output = PARTITION(supplier, name="part", by=(region_name, supplier_name))(
    region_name,
    supplier_name,
    total_sales=SUM(part.lines.extended_price * (1 - part.lines.discount))
).TOP_K(3, by=total_sales.DESC())

pydough_output=pydough.to_df(output)
pydough_output

,region_name,supplier_name,total_sales
0,AFRICA,Supplier#000005994,28698575.631300
1,EUROPE,Supplier#000006490,28579287.000400
2,MIDDLE EAST,Supplier#000005991,28542342.998400


Compare results in SQL and PyDough:

In [18]:
dfcompare.compare_df(pydough_output, sql_output, query_category="", 
                     question="")

True

PyDough to SQL:

In [19]:
%%pydough

pydough.to_sql(output)

'SELECT region_name, supplier_name, total_sales FROM (SELECT supplier_name, region_name, total_sales, ordering_1 FROM (SELECT supplier_name, region_name, COALESCE(agg_0, 0) AS total_sales, COALESCE(agg_0, 0) AS ordering_1 FROM (SELECT _table_alias_6.supplier_name AS supplier_name, _table_alias_6.region_name AS region_name, agg_0 FROM (SELECT supplier_name, region_name FROM (SELECT name_3 AS region_name, name AS supplier_name FROM (SELECT s_name AS name, s_nationkey AS nation_key FROM main.SUPPLIER) LEFT JOIN (SELECT _table_alias_0.key AS key, name AS name_3 FROM (SELECT n_nationkey AS key, n_regionkey AS region_key FROM main.NATION) AS _table_alias_0 INNER JOIN (SELECT r_name AS name, r_regionkey AS key FROM main.REGION) AS _table_alias_1 ON region_key = _table_alias_1.key) ON nation_key = key) GROUP BY supplier_name, region_name) AS _table_alias_6 LEFT JOIN (SELECT region_name, supplier_name, SUM(extended_price * (1 - discount)) AS agg_0 FROM (SELECT region_name, supplier_name, discou

## Bussines Context 2

This query seeks to analyze which are the five customers that return the most orders. The idea is to find the name of the customer, the segment to which it belongs and the total of returned orders.

SQL:

In [ ]:
query = '''
SELECT
    C_NAME AS customer_name,
    C_MKTSEGMENT AS market_segment,
    COUNT(*) AS return_count
FROM
    CUSTOMER
JOIN
    ORDERS ON C_CUSTKEY = O_CUSTKEY
JOIN
    LINEITEM ON O_ORDERKEY = L_ORDERKEY
WHERE
    L_RETURNFLAG = 'R'  -- 'R' indicates a returned/cancelled item
GROUP BY
    C_NAME, C_MKTSEGMENT
ORDER BY
    return_count DESC
LIMIT 5;
'''

sql_output = pd.read_sql_query(query, connection)
sql_output

,customer_name,market_segment,return_count
0,Customer#000050605,HOUSEHOLD,60
1,Customer#000121909,AUTOMOBILE,60
2,Customer#000021016,MACHINERY,57
3,Customer#000042619,MACHINERY,57
4,Customer#000075160,HOUSEHOLD,57


PyDough solution:

In [28]:
%%pydough

output = PARTITION(customers, name="cust", by=(name, mktsegment))(
    customer_name=name,
    market_segment=mktsegment,
    return_count=COUNT(cust.orders.lines.WHERE(return_flag == "R"))
).TOP_K(5, by=return_count.DESC())

pydough_output=pydough.to_df(output)
pydough_output

,customer_name,market_segment,return_count
0,Customer#000121909,AUTOMOBILE,60
1,Customer#000050605,HOUSEHOLD,60
2,Customer#000075160,HOUSEHOLD,57
3,Customer#000021016,MACHINERY,57
4,Customer#000042619,MACHINERY,57


Compare results in SQL and PyDough:

In [29]:
dfcompare.compare_df(pydough_output, sql_output, query_category="", 
                     question="")

np.True_

PyDough to SQL:

In [30]:
%%pydough

pydough.to_sql(output)

"SELECT customer_name, market_segment, return_count FROM (SELECT customer_name, market_segment, return_count, ordering_1 FROM (SELECT name AS customer_name, mktsegment AS market_segment, COALESCE(agg_0, 0) AS return_count, COALESCE(agg_0, 0) AS ordering_1 FROM (SELECT _table_alias_2.mktsegment AS mktsegment, _table_alias_2.name AS name, agg_0 FROM (SELECT mktsegment, name FROM (SELECT c_name AS name, c_mktsegment AS mktsegment FROM main.CUSTOMER) GROUP BY mktsegment, name) AS _table_alias_2 LEFT JOIN (SELECT name, mktsegment, COUNT() AS agg_0 FROM (SELECT name, mktsegment FROM (SELECT name, mktsegment, return_flag FROM (SELECT name, mktsegment, _table_alias_1.key AS key_2 FROM (SELECT c_name AS name, c_custkey AS key, c_mktsegment AS mktsegment FROM main.CUSTOMER) AS _table_alias_0 INNER JOIN (SELECT o_orderkey AS key, o_custkey AS customer_key FROM main.ORDERS) AS _table_alias_1 ON _table_alias_0.key = customer_key) INNER JOIN (SELECT l_returnflag AS return_flag, l_orderkey AS order_k

## Bussines Context 3

The idea is to select the 20 customers who have bought the most according to the money invested. The customer is identified by his id and name, and we also want to know the nation and region to which he belongs, as well as the total number of orders and how much he has spent on them. 

SQL:

In [50]:
query = '''
WITH CustomerPurchases AS (
    SELECT 
        O_CUSTKEY, 
        COUNT(DISTINCT O_ORDERKEY) AS TotalOrders, 
        SUM(L_EXTENDEDPRICE * (1 - L_DISCOUNT)) AS TotalSpent
    FROM ORDERS
    JOIN LINEITEM ON O_ORDERKEY = L_ORDERKEY
    GROUP BY O_CUSTKEY
), CustomerInfo AS (
    SELECT 
        C_CUSTKEY, 
        C_NAME,
        N_NAME AS Nation,
        R_NAME AS Region
    FROM CUSTOMER
    JOIN NATION ON C_NATIONKEY = N_NATIONKEY
    JOIN REGION ON N_REGIONKEY = R_REGIONKEY
)
SELECT 
    C.C_CUSTKEY AS customer_id, 
    C.C_NAME AS customer_name, 
    C.Nation, 
    C.Region, 
    P.TotalOrders,
    P.TotalSpent
FROM CustomerPurchases P
JOIN CustomerInfo C ON P.O_CUSTKEY = C.C_CUSTKEY
ORDER BY P.TotalSpent DESC
LIMIT 20;
'''

sql_output = pd.read_sql_query(query, connection)
sql_output

,customer_id,customer_name,Nation,Region,TotalOrders,TotalSpent
0,143500,Customer#000143500,IRAN,MIDDLE EAST,39,6757566.021800
1,95257,Customer#000095257,BRAZIL,AMERICA,36,6294115.334000
2,87115,Customer#000087115,KENYA,AFRICA,34,6184649.517600
3,131113,Customer#000131113,ETHIOPIA,AFRICA,37,6080943.830500
4,134380,Customer#000134380,ALGERIA,AFRICA,37,6075141.963500
5,103834,Customer#000103834,IRAQ,MIDDLE EAST,31,6059770.323200
6,69682,Customer#000069682,MOZAMBIQUE,AFRICA,39,6057779.034800
7,102022,Customer#000102022,INDONESIA,ASIA,41,6039653.633500
8,98587,Customer#000098587,CHINA,ASIA,37,6027021.585500
9,64660,Customer#000064660,MOZAMBIQUE,AFRICA,31,5905659.615900


PyDough solution:

In [51]:
%%pydough
output = customers(
    customer_id=key,
    customer_name=name,
    Nation=nation.name,
    Region=nation.region.name,
    TotalOrders=COUNT(orders),
    TotalSpent=SUM(orders.lines.extended_price * (1 - orders.lines.discount))
).TOP_K(20, TotalSpent.DESC())

pydough_output=pydough.to_df(output)
pydough_output

,customer_id,customer_name,Nation,Region,TotalOrders,TotalSpent
0,143500,Customer#000143500,IRAN,MIDDLE EAST,39,6757566.021800
1,95257,Customer#000095257,BRAZIL,AMERICA,36,6294115.334000
2,87115,Customer#000087115,KENYA,AFRICA,34,6184649.517600
3,131113,Customer#000131113,ETHIOPIA,AFRICA,37,6080943.830500
4,134380,Customer#000134380,ALGERIA,AFRICA,37,6075141.963500
5,103834,Customer#000103834,IRAQ,MIDDLE EAST,31,6059770.323200
6,69682,Customer#000069682,MOZAMBIQUE,AFRICA,39,6057779.034800
7,102022,Customer#000102022,INDONESIA,ASIA,41,6039653.633500
8,98587,Customer#000098587,CHINA,ASIA,37,6027021.585500
9,64660,Customer#000064660,MOZAMBIQUE,AFRICA,31,5905659.615900


Compare results in SQL and PyDough:

In [52]:
dfcompare.compare_df(pydough_output, sql_output, query_category="", 
                     question="")

True

PyDough to SQL:

In [53]:
%%pydough

pydough.to_sql(output)

'SELECT customer_id, customer_name, Nation, Region, TotalOrders, TotalSpent FROM (SELECT Nation, Region, TotalOrders, TotalSpent, customer_id, customer_name, ordering_2 FROM (SELECT name_3 AS Nation, name_6 AS Region, COALESCE(agg_0, 0) AS TotalOrders, COALESCE(agg_1, 0) AS TotalSpent, key AS customer_id, name AS customer_name, COALESCE(agg_1, 0) AS ordering_2 FROM (SELECT name, key, name_3, name_6, agg_0, agg_1 FROM (SELECT name, key, name_3, name_6, agg_0 FROM (SELECT name, _table_alias_4.key AS key, name_3, name_6 FROM (SELECT _table_alias_0.name AS name, _table_alias_0.key AS key, nation_key, _table_alias_1.name AS name_3 FROM (SELECT c_name AS name, c_custkey AS key, c_nationkey AS nation_key FROM main.CUSTOMER) AS _table_alias_0 LEFT JOIN (SELECT n_name AS name, n_nationkey AS key FROM main.NATION) AS _table_alias_1 ON nation_key = _table_alias_1.key) AS _table_alias_4 LEFT JOIN (SELECT _table_alias_2.key AS key, name AS name_6 FROM (SELECT n_nationkey AS key, n_regionkey AS regi

## Bussines Context 4

Average discount applied on orders for each market segment. 
This query allows you to analyze which market segments receive higher discounts on their orders. This can help assess whether discount strategies are aligned with business objectives and whether certain segments are getting too high or too low discounts compared to others.

SQL:

In [6]:
query = '''
SELECT 
    C.C_MKTSEGMENT AS MarketSegment,
    AVG(L.L_DISCOUNT) * 100 AS AvgDiscountPercentage
FROM CUSTOMER C
JOIN ORDERS O ON C.C_CUSTKEY = O.O_CUSTKEY
JOIN LINEITEM L ON O.O_ORDERKEY = L.L_ORDERKEY
GROUP BY C.C_MKTSEGMENT
ORDER BY AvgDiscountPercentage DESC;
'''

sql_output = pd.read_sql_query(query, connection)
sql_output

,MarketSegment,AvgDiscountPercentage
0,MACHINERY,5.006444
1,HOUSEHOLD,5.000242
2,FURNITURE,4.999733
3,AUTOMOBILE,4.998568
4,BUILDING,4.994801


PyDough solution:

In [12]:
%%pydough

output = PARTITION(customers, name="part", by=mktsegment)(
    MarketSegment=mktsegment,
    AvgDiscountPercentage=AVG(part.orders.lines.discount)*100
).ORDER_BY(AvgDiscountPercentage.DESC())

pydough_output=pydough.to_df(output)
pydough_output

,MarketSegment,AvgDiscountPercentage
0,MACHINERY,5.006444
1,HOUSEHOLD,5.000242
2,FURNITURE,4.999733
3,AUTOMOBILE,4.998568
4,BUILDING,4.994801


Compare results in SQL and PyDough:

In [13]:
dfcompare.compare_df(pydough_output, sql_output, query_category="", 
                     question="")

True

PyDough to SQL:

In [14]:
%%pydough

pydough.to_sql(output)

'SELECT MarketSegment, AvgDiscountPercentage FROM (SELECT agg_0 * 100 AS AvgDiscountPercentage, mktsegment AS MarketSegment, agg_0 * 100 AS ordering_1 FROM (SELECT _table_alias_2.mktsegment AS mktsegment, agg_0 FROM (SELECT mktsegment FROM (SELECT c_mktsegment AS mktsegment FROM main.CUSTOMER) GROUP BY mktsegment) AS _table_alias_2 LEFT JOIN (SELECT mktsegment, AVG(discount) AS agg_0 FROM (SELECT mktsegment, discount FROM (SELECT mktsegment, _table_alias_1.key AS key_2 FROM (SELECT c_custkey AS key, c_mktsegment AS mktsegment FROM main.CUSTOMER) AS _table_alias_0 INNER JOIN (SELECT o_orderkey AS key, o_custkey AS customer_key FROM main.ORDERS) AS _table_alias_1 ON _table_alias_0.key = customer_key) INNER JOIN (SELECT l_discount AS discount, l_orderkey AS order_key FROM main.LINEITEM) ON key_2 = order_key) GROUP BY mktsegment) AS _table_alias_3 ON _table_alias_2.mktsegment = _table_alias_3.mktsegment)) ORDER BY ordering_1 DESC'

## Bussines Context 5

The query wants to find the cheapest suppliers for the 10 best selling products. It is important to know the name of the product and the total solds, the supplier name and nation, and the price at which it is supplied. 

SQL:

In [ ]:
query = '''
WITH TopProducts AS (
    -- Obtener los 10 productos más vendidos en cantidad total
    SELECT 
        L_PARTKEY AS PartKey,
        P_PARTKEY AS ProductId,
        P_NAME AS ProductName,
        SUM(L_QUANTITY) AS TotalSold
    FROM LINEITEM
    JOIN PART ON L_PARTKEY = P_PARTKEY
    GROUP BY L_PARTKEY, P_NAME, P_PARTKEY
    ORDER BY TotalSold DESC
    LIMIT 10
), RankedSuppliers AS (
    -- Encontrar el proveedor más económico para cada uno de los 10 productos más vendidos
    SELECT 
        TP.PartKey,
        TP.ProductName,
        TP.ProductId,
        TP.TotalSold,  -- Se mantiene la cantidad total vendida
        S.S_NAME AS SupplierName,
        N.N_NAME AS SupplierNation,
        PS.PS_SUPPLYCOST AS SupplyCost,
        RANK() OVER (PARTITION BY PS.PS_PARTKEY ORDER BY PS.PS_SUPPLYCOST ASC) AS CostRank
    FROM PARTSUPP PS
    JOIN SUPPLIER S ON PS.PS_SUPPKEY = S.S_SUPPKEY
    JOIN NATION N ON S.S_NATIONKEY = N.N_NATIONKEY
    JOIN TopProducts TP ON PS.PS_PARTKEY = TP.PartKey  -- Unimos con los productos más vendidos
)
SELECT 
    ProductId,
    ProductName,
    TotalSold,  -- Agregamos la cantidad total vendida en el resultado final
    SupplierName,
    SupplierNation,
    SupplyCost
FROM RankedSuppliers
WHERE CostRank = 1
ORDER BY TotalSold DESC;  -- Ordenamos por cantidad vendida para mayor claridad
'''

sql_output = pd.read_sql_query(query, connection)
sql_output

## Bussines Context 6

Evaluate the performance of suppliers in terms of quantity of products supplied in a specific period. This allows to identify which suppliers are more consistent in supplying the company, helping in decision making for strategic commercial relationships. The year selected is 1998. Return the top 10 suppliers. 

SQL:

In [ ]:
query = '''
WITH SupplierPerformance AS (
    -- Calculamos la cantidad total de productos suministrados por cada proveedor en un período específico
    SELECT 
        S.S_SUPPKEY AS SupplierKey,
        S.S_NAME AS SupplierName,
        N.N_NAME AS SupplierNation,
        SUM(PS.PS_AVAILQTY) AS TotalSupplied
    FROM PARTSUPP PS
    JOIN SUPPLIER S ON PS.PS_SUPPKEY = S.S_SUPPKEY
    JOIN NATION N ON S.S_NATIONKEY = N.N_NATIONKEY
    WHERE PS.PS_PARTKEY IN (
        -- Filtramos los productos que han sido ordenados en 1998
        SELECT DISTINCT L.L_PARTKEY
        FROM LINEITEM L
        JOIN ORDERS O ON L.L_ORDERKEY = O.O_ORDERKEY
        WHERE O.O_ORDERDATE BETWEEN '1998-01-01' AND '1998-12-31'
    )
    GROUP BY S.S_SUPPKEY, S.S_NAME, N.N_NAME
)
SELECT 
    SupplierName,
    SupplierNation,
    TotalSupplied
FROM SupplierPerformance
ORDER BY TotalSupplied DESC
LIMIT 10;  -- Muestra los 10 proveedores más confiables en términos de volumen
'''

sql_output = pd.read_sql_query(query, connection)
sql_output

,SupplierName,SupplierNation,TotalSupplied
0,Supplier#000008099,INDIA,491764
1,Supplier#000004527,CANADA,463634
2,Supplier#000007747,KENYA,463045
3,Supplier#000002621,GERMANY,459166
4,Supplier#000005322,FRANCE,458062
5,Supplier#000003467,GERMANY,457830
6,Supplier#000007181,EGYPT,453131
7,Supplier#000006638,ALGERIA,452991
8,Supplier#000004592,FRANCE,452843
9,Supplier#000006048,IRAQ,452323


PyDough solution:

In [37]:
%%pydough

selected_lines = lines(
    SupplierName=supplier.name,
    SupplierNation=supplier.nation.name
)

output = PARTITION(selected_lines, name="p", by=(SupplierName, SupplierNation))(
    SupplierName,
    SupplierNation,
    TotalSupplied=COUNT(p.part_and_supplier.availqty)
).TOP_K(10, TotalSupplied.ASC())

pydough_output=pydough.to_df(output)
pydough_output

,SupplierName,SupplierNation,TotalSupplied
0,Supplier#000007905,ETHIOPIA,517
1,Supplier#000001364,KENYA,523
2,Supplier#000001128,IRAN,524
3,Supplier#000004435,PERU,524
4,Supplier#000008137,KENYA,524
5,Supplier#000001084,ARGENTINA,526
6,Supplier#000007038,FRANCE,526
7,Supplier#000006700,VIETNAM,528
8,Supplier#000003516,PERU,529
9,Supplier#000003730,INDONESIA,529


## Bussines Context 7

This query seeks to identify products that have been sold by more than 5 different suppliers, which allows to detect those with high competition in the market. If a product has multiple suppliers, there is likely to be pressure on prices and profit margins, which can impact both buyers and sellers.

SQL:

In [53]:
query = '''
SELECT 
    P.P_NAME AS ProductName,
    COUNT(DISTINCT S.S_SUPPKEY) AS UniqueSuppliers
FROM PART P
JOIN PARTSUPP PS ON P.P_PARTKEY = PS.PS_PARTKEY
JOIN SUPPLIER S ON PS.PS_SUPPKEY = S.S_SUPPKEY
JOIN LINEITEM L ON P.P_PARTKEY = L.L_PARTKEY
GROUP BY P.P_NAME
HAVING UniqueSuppliers > 5
ORDER BY UniqueSuppliers DESC;
'''

sql_output = pd.read_sql_query(query, connection)
sql_output

,ProductName,UniqueSuppliers
0,blush burlywood red blanched olive,8
1,azure lime rosy peru powder,8
2,aquamarine frosted tomato medium navy,8


PyDough solution:

Structure of PyDough graph: TPCH

  customers
  ├── acctbal
  ├── address
  ├── comment
  ├── key
  ├── mktsegment
  ├── name
  ├── nation_key
  ├── phone
  ├── nation [one member of nations] (reverse of nations.customers)
  └── orders [multiple orders] (reverse of orders.customer)

  lines
  ├── comment
  ├── commit_date
  ├── discount
  ├── extended_price
  ├── line_number
  ├── order_key
  ├── part_key
  ├── quantity
  ├── receipt_date
  ├── return_flag
  ├── ship_date
  ├── ship_instruct
  ├── ship_mode
  ├── status
  ├── supplier_key
  ├── tax
  ├── order [one member of orders] (reverse of orders.lines)
  ├── part [one member of parts] (reverse of parts.lines)
  ├── part_and_supplier [one member of supply_records] (reverse of supply_records.lines)
  └── supplier [one member of suppliers] (reverse of suppliers.lines)

  nations
  ├── comment
  ├── key
  ├── name
  ├── region_key
  ├── customers [multiple customers] (reverse of customers.nation)
  ├── region [one member of regions] (reverse of regions.nations)
  └── suppliers [multiple suppliers] (reverse of suppliers.nation)

  orders
  ├── clerk
  ├── comment
  ├── customer_key
  ├── key
  ├── order_date
  ├── order_priority
  ├── order_status
  ├── ship_priority
  ├── total_price
  ├── customer [one member of customers] (reverse of customers.orders)
  └── lines [multiple lines] (reverse of lines.order)

  parts
  ├── brand
  ├── comment
  ├── container
  ├── key
  ├── manufacturer
  ├── name
  ├── part_type
  ├── retail_price
  ├── size
  ├── lines [multiple lines] (reverse of lines.part)
  └── supply_records [multiple supply_records] (reverse of supply_records.part)

  regions
  ├── comment
  ├── key
  ├── name
  └── nations [multiple nations] (reverse of nations.region)

  suppliers
  ├── account_balance
  ├── address
  ├── comment
  ├── key
  ├── name
  ├── nation_key
  ├── phone
  ├── lines [multiple lines] (reverse of lines.supplier)
  ├── nation [one member of nations] (reverse of nations.suppliers)
  └── supply_records [multiple supply_records] (reverse of supply_records.supplier)

  supply_records
  ├── availqty
  ├── comment
  ├── part_key
  ├── supplier_key
  ├── supplycost
  ├── lines [multiple lines] (reverse of lines.part_and_supplier)
  ├── part [one member of parts] (reverse of parts.supply_records)
  └── supplier [one member of suppliers] (reverse of suppliers.supply_records)